this is the second part, and it uses files from first run of centrostruct.py module


as result we should have tab with line azimuths, line angles, structure azimuth and rotations. 


i am also going to add xarms here - sizes and angles

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString
import csv
from math import sqrt, acos, degrees

In [2]:
# пути
workdir = Path(r'D:\work\2022_m\110_man-nps1\test')
resultdir = workdir / 'result'
tempdir = workdir / 'temp'

some functions for calc

In [3]:
# from math import sqrt, acos, degrees

def azimuth(a, b):
    # by two points we get azimuth
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dist = round(sqrt(dx*dx + dy*dy), 2)    # dist a to b
    dx2 = abs(dx)
    beta = degrees(acos(dx2/dist))
    if dx > 0:
        if dy < 0:
            angle = 270 + beta
        else:
            angle = 270 - beta
    else:
        if dy < 0:
            angle = 90 - beta
        else:
            angle = 90 + beta

    return round(angle, 1), dist

In [4]:
# import csv

def xyz_read(path):
    # read xyz arrays from files
    with open(path, newline='') as xyz_file:
        spamlist = []
        spamreader = csv.reader(xyz_file, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC, skipinitialspace=True)
        for row in spamreader:
            spamlist.append(row)

    return spamlist

In [5]:
def xarms_levels(array, segment_size):
    segmented = []
    start = min(array)
    end = start + segment_size
    while end < max(array):
        segmented.append(sum(end > i > start for i in np_str_z))
        start += segment_size
        end += segment_size

        
    # then choose only bigget than mean*2
    ma = []
    for nu, i in enumerate(segmented):
        if i > (np.mean(segmented)*2):
            ma.append((nu, i))

            
    # then group them if they close to each other
    ma2 = {}
    low_level = 0
    points_sum = 0

    for i in range(len(ma)):
        # если следующий сегмент также отобран
        if i != len(ma)-1 and ma[i][0]+1 == (ma[i+1][0]):
            points_sum += ma[i+1][1]

            # если предыдущий сегмент не был отобран 
            if i > 0 and not ma[i][0]-1 == (ma[i-1][0]):
                points_sum += ma[i][1]
                low_level = ma[i][0]

            ma2[low_level] = points_sum

        else:
            #print('skip')
            low_level = 0
            points_sum = 0

            
    #  из отобранных групп выделяем три наибольшие
    # такой отбор не стаботает на траверсах постоянной высоты - там будет по два выброса на траверсу
    selected = sorted(ma2.items(), key=lambda item: item[1])[-3:]
    #print(selected)

    h_trav = []   # нижие кромки траверс
    for i in selected:
        h_trav.append(round((min(array) + segment_size*i[0]), 1))
    #print(sorted(h_trav))    
    
    return sorted(h_trav)


In [6]:
def midpoint(p1, p2):
    # calc middle point
    mx = round((p1[0]+p2[0])/2, 2)
    my = round((p1[1]+p2[1])/2, 2)
    
    return (mx, my)


firstly get the initial table 

In [7]:
cgt_tab = pd.read_csv(workdir / '110_v2.txt', delimiter='\t', index_col='index', encoding = 'cp1251')
# cgt_tab = pd.read_excel(resultdir / 'cgtw_output.xlsx', index_col=0)

In [8]:
cgt_tab[:3]

,c_line,id,x,y,z,x_bot,y_bot,z_bot,x_top,y_top,z_top,dist
index,,,,,,,,,,,,
1,1,ПС_Мангазея,557438.02,7388310.15,34.22,557439.52,7388308.80,19.58,557439.76,7388308.81,34.24,2.02
2,1,1,557468.82,7388296.03,45.72,557468.30,7388296.38,18.98,557468.44,7388296.28,45.81,0.63
3,1,2,557639.86,7388331.97,50.92,557639.36,7388332.05,19.05,557639.43,7388332.15,51.48,0.51


create new tab based of initial

In [9]:
ang_tab = cgt_tab[['c_line', 'id']].copy()

In [10]:
# add new cols
new_cols = ['span name', 'span length 2d', 'span azimuth', 'line angle', 'structure azimuth', 'structure rotation', 'x_arm start']
for col in new_cols:
    ang_tab[col] = np.nan

In [11]:
ang_tab[:3]

,c_line,id,span name,span length 2d,span azimuth,line angle,structure azimuth,structure rotation,x_arm start
index,,,,,,,,,
1,1,ПС_Мангазея,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


find span name, length, azimuth, line angle

In [12]:
for n in range(len(cgt_tab)-1):
    idx = cgt_tab.iloc[n].name  # find index and work with index
    
    if cgt_tab.loc[idx, 'c_line'] == cgt_tab.loc[idx+1, 'c_line']:
        # check if we on the same c-line
        ang_tab.loc[idx, 'span name'] = f"{cgt_tab.loc[idx, 'id']} - {cgt_tab.loc[idx+1, 'id']}"  # span id 
        a = (cgt_tab.loc[idx, 'x_bot'], cgt_tab.loc[idx, 'y_bot'])   # start point
        b = (cgt_tab.loc[idx+1, 'x_bot'], cgt_tab.loc[idx+1, 'y_bot'])   # end point
        ang_tab.loc[idx, 'span azimuth'], ang_tab.loc[idx, 'span length 2d'] = azimuth(a, b)
        if idx > 1:
            if cgt_tab.loc[idx, 'c_line'] == cgt_tab.loc[idx-1, 'c_line']:
                spam = ang_tab.loc[idx, 'span azimuth'] - ang_tab.loc[idx-1, 'span azimuth']
                if spam < -180:
                    ang_tab.loc[idx, 'line angle'] = spam + 360
                else:
                    ang_tab.loc[idx, 'line angle'] = spam
            else:
                ang_tab.loc[idx, 'line angle'] = 0
        else:
                ang_tab.loc[idx, 'line angle'] = 0
    
    #print(span_name, azim, span_len)

In [13]:
ang_tab

,c_line,id,span name,span length 2d,span azimuth,line angle,structure azimuth,structure rotation,x_arm start
index,,,,,,,,,
1,1,ПС_Мангазея,ПС_Мангазея - 1,31.35,113.4,0.0,NaN,NaN,NaN
2,1,1,1 - 2,174.74,78.2,-35.2,NaN,NaN,NaN
3,1,2,2 - 3,256.40,78.2,0.0,NaN,NaN,NaN
4,1,3,3 - 4,232.62,102.1,23.9,NaN,NaN,NaN
5,1,4,4 - 5,110.09,102.1,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
196,1,195,195 - ПС_НПС-1,34.57,13.5,-61.5,NaN,NaN,NaN
197,1,ПС_НПС-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,2,4,4 - 4a,75.02,143.7,0.0,NaN,NaN,NaN


lets work with x-arms

firstly  will find the start level - the lowest xarm on structure

for this work we need to upload ALS structure points - we will use created on the first step

In [15]:
# we should go for initial table and select each str again

# here in notepad we will work with first structure then all the script will work as separate py file
#for n in range(len(cgt_tab)):
#    idx = cgt_tab.iloc[n].name  # find index and work with index
#    tow_cut = xyz_read(tempdir / str(f"{idx}_{cgt_tab.loc[idx, 'id']}_str.xyz"))

idx = 13    # work with one structure only now
tow_cut = xyz_read(tempdir / str(f"{idx}_{cgt_tab.loc[idx, 'id']}_str.xyz"))

In [16]:
np_str = np.array(tow_cut)
np_str_z = np_str[:,2]    # leave only z-coord
str_hight = round(max(np_str_z)-min(np_str_z), 2)
np_str_z = np.delete(np_str_z, np_str_z<str_hight+4, 0)   # cut 4 m from grd

In [53]:
[z1, z2, z3] = xarms_levels(np_str_z, 0.1)   # find xarm levels
z1, z2, z3

(42.7, 46.7, 50.7)

In [18]:
# cut the legs again (more accurate)
np_str_cut = np.delete(np_str, np_str[:,2]<(z1-0.1), 0)

# now work with top part
np_str_mp = MultiPoint(np_str_cut)    # to multipoint (needs to shapely)
rect = np_str_mp.minimum_rotated_rectangle
rect_c = rect.centroid

In [54]:
# find a longest side and azimuth

side_a = azimuth(list(rect.exterior.coords)[0], list(rect.exterior.coords)[1])
side_b = azimuth(list(rect.exterior.coords)[1], list(rect.exterior.coords)[2])
if side_a[1] > side_b[1]:
    str_azmt = side_a[0]
    axl = LineString([midpoint(list(rect.exterior.coords)[0], list(rect.exterior.coords)[3]), 
                     midpoint(list(rect.exterior.coords)[1], list(rect.exterior.coords)[2])])
else:
    str_azmt = side_b[0]
    axl = LineString([midpoint(list(rect.exterior.coords)[0], list(rect.exterior.coords)[1]), 
                     midpoint(list(rect.exterior.coords)[2], list(rect.exterior.coords)[3])])


# def midpoint(p1, p2):
#     # calc middle point
#     mx = round((p1[0]+p2[0])/2, 2)
#     my = round((p1[1]+p2[1])/2, 2)
    
#     return (mx, my)

print(axl.length)
print(axl)
print(str_azmt)

8.701315992757333
LINESTRING (559670.92 7387998.85, 559673.19 7388007.25)
15.1


In [29]:
axl.coords[1][0]


557444.31

In [51]:
def group_arms(levels, segment_size):
    # group them if they close to each other
    
    new_levels = [levels[0]]  # add first one
    for i in range(1, len(levels)):
        if levels[i-1] + segment_size < levels[i]:
            new_levels.append(levels[i])

        


#         if levels[i] == round((levels[i+1] + segment_size), 1):
#             pass
#         else:
#             new_levels.append(levels[i])
#             spam = levels[i]"

    return new_levels

In [21]:
aa = [42.8, 46.7, 46.8, 50.8, 53.7]

In [52]:
group_arms(aa, 0.3)

[42.8, 46.7, 50.8, 53.7]

In [74]:
def az_str(az1, az2):
    az1 = (az1+az2)/2 
    az2 = az1-90
    if az2 < 0: 
        az2 += 360
    elif az2 > 360: 
        az2 -= 360
        
    return az1, az2

In [82]:
az_str(327, 250)

(288.5, 198.5)

In [107]:
aaa = [[3, 1]]

In [109]:
aaa[0][1]

1

In [106]:
sorted(aaa)[-2:]

[[3, 1], [5, 2]]

In [116]:
def hor_axes(a_tab):
    # values
    color = 0
    text = ""   # future part of DXF with hor axes
    structures = []   # list of str to exclude dubs

    for n in range(len(a_tab)):
        idx = a_tab.iloc[n].name   # find index and work with it

        spam = a_tab.loc[idx, 'id']   # structure ID
        # check if it not in list
        if spam not in structures:
            structures.append(spam)

            for arm_name in ('x_arm 1', 'x_arm 2', 'x_arm 3'):
                # create 3 layers with xarms
                xarm = a_tab.loc[idx, arm_name]
                if not pd.isna(xarm): 
                    # convert string to float
                    xarm = xarm.split(',')
                    #for i in range(len(xarm)):
                    #    xarm[i] = float(xarm[i].strip('( )'))

                    layername = arm_name

                    # add header
                    text += f'0\nLINE\n 8\n{layername}\n 62\n     {color}\n'

                    # add first coords
                    text += f" 10\n{xarm[0]}\n 20\n{xarm[1]}\n 30\n{xarm[2]}\n"

                    # add second coords
                    text += f" 11\n{xarm[3]}\n 21\n{xarm[4]}\n 31\n{xarm[5]}\n"

    return text

In [111]:
ang_tab2 = pd.read_excel(workdir / 'ang_tab.xlsx', index_col=0)

In [112]:
ang_tab2

,c_line,id,span name,span length 2d,span azimuth,line angle,structure azimuth,structure rotation,x_arm start,x_arm 1,x_arm 2,x_arm 3
index,,,,,,,,,,,,
1,1,ПС_Мангазея,ПС_Мангазея - 1,31.35,113.4,0.0,NaN,NaN,31.3,"((557443.52, 7388317.64, 31.1), (557436.190000...",NaN,NaN
2,1,1,1 - 2,174.74,78.2,-35.2,NaN,NaN,31.5,"((557471.4600000001, 7388300.11, 31.4), (55746...","((557471.9, 7388301.55, 35.35), (557468.690000...","((557471.5800000001, 7388300.1, 39.37), (55746..."
3,1,2,2 - 3,256.40,78.2,0.0,NaN,NaN,43.5,"((557638.28, 7388336.17, 43.46), (557640.24, 7...","((557638.86, 7388334.29, 47.48), (557639.7, 73...",NaN
4,1,3,3 - 4,232.62,102.1,23.9,NaN,NaN,35.9,"((557891.5399999999, 7388388.01, 35.9), (55788...","((557891.76, 7388389.48, 39.85), (557890.15, 7...","((557891.8099999999, 7388388.15, 43.77), (5578..."
5,1,4,4 - 5,110.09,102.1,0.0,NaN,NaN,29.2,"((558119.4800000001, 7388339.36, 29.15), (5581...","((558119.53, 7388340.71, 33.03), (558116.51, 7...","((558120.6900000001, 7388338.8, 36.94), (55811..."
...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,195,195 - ПС_НПС-1,34.57,13.5,-61.5,NaN,NaN,26.2,"((572287.59, 7359971.12, 26.1), (572295.74, 73...","((572283.73, 7359970.4799999995, 32.4), (57229...","((572285.66, 7359969.24, 38.92), (572295.33, 7..."
197,1,ПС_НПС-1,NaN,NaN,NaN,NaN,NaN,NaN,27.9,NaN,NaN,NaN
198,2,4,4 - 4a,75.02,143.7,0.0,NaN,NaN,29.2,NaN,NaN,NaN


In [117]:
hor_axes(ang_tab2)

yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay
yay


'0\nLINE\n 8\nx_arm 1\n 62\n     0\n 10\n((557443.52\n 20\n 7388317.64\n 30\n 31.1)\n 11\n (557436.1900000001\n 21\n 7388300.39\n 31\n 31.07))\n0\nLINE\n 8\nx_arm 1\n 62\n     0\n 10\n((557471.4600000001\n 20\n 7388300.11\n 30\n 31.4)\n 11\n (557468.67\n 21\n 7388292.42\n 31\n 31.4))\n0\nLINE\n 8\nx_arm 2\n 62\n     0\n 10\n((557471.9\n 20\n 7388301.55\n 30\n 35.35)\n 11\n (557468.6900000001\n 21\n 7388291.02\n 31\n 35.34))\n0\nLINE\n 8\nx_arm 3\n 62\n     0\n 10\n((557471.5800000001\n 20\n 7388300.1\n 30\n 39.37)\n 11\n (557468.51\n 21\n 7388292.43\n 31\n 39.38))\n0\nLINE\n 8\nx_arm 1\n 62\n     0\n 10\n((557638.28\n 20\n 7388336.17\n 30\n 43.46)\n 11\n (557640.24\n 21\n 7388327.819999999\n 31\n 43.49))\n0\nLINE\n 8\nx_arm 2\n 62\n     0\n 10\n((557638.86\n 20\n 7388334.29\n 30\n 47.48)\n 11\n (557639.7\n 21\n 7388329.88\n 31\n 47.4))\n0\nLINE\n 8\nx_arm 1\n 62\n     0\n 10\n((557891.5399999999\n 20\n 7388388.01\n 30\n 35.9)\n 11\n (557887.83\n 21\n 7388380.55\n 31\n 35.8))\n0\nLINE\n